In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('always')
from sklearn.model_selection import train_test_split
import shapely.speedups
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
import re
from itertools import combinations
import random
import joblib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
#
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
def plot_regression(modelo,x,y,title="",xlabel="x",ylabel="y"):
    plt.figure()
    
    plt.plot(x,y,"o",label="Valores verdaderos")
    plt.plot(x,modelo.predict(x),"x",label="Valores estimados")
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_test.csv').copy()
viviendas = pd.concat([viviendas_train, viviendas_test])

In [ ]:
viviendas = pd.get_dummies(viviendas, columns = ["property_type"])
viviendas = pd.get_dummies(viviendas, columns = ["barrio"])

In [ ]:
viviendas_train, viviendas_test = train_test_split(viviendas, test_size=0.2, random_state=2)

definimos las columnas que utilizaremos para entrenar las redes

In [ ]:
columnas_predictoras=viviendas_train.columns.to_list()
for variable in ['property_price', 'start_date', 'end_date', 'property_title', 'geometry']:
    columnas_predictoras.remove(variable)
columnas_predictoras

## RANDOMIZADOR


Para ontrenar las redes neuronales utilizaresmos un random_search para optimizar los hiperparametros

In [ ]:
a = ['property_price', 'start_date', 'end_date', 'property_title', 'geometry']
b = viviendas_train.loc[:,a]
b

In [ ]:
#al instanciar un ModeloRedes crea y entrena una red neuronal con los hiperparametros pasados
class ModeloRedes:
    def __init__(self, dataset_train, dataset_test, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size):
        self.variables_predictoras = variables_predictoras
        self.cantidad_primera_capa = cantidad_de_primera_capa
        self.funcion_activacion = funcion_activacion
        self.optimizador = optimizador
        self.learning_rate = learning_rate
        self.resultado = None
        self.metrica = metrica
        self.epoch = epoch
        self.batch_size = batch_size
        self.modelo = None

        
        self.x_test = dataset_test.loc[:,variables_predictoras]
        self.y_test = dataset_test.loc[:,'property_price']
        self.x_train = dataset_train.loc[:,variables_predictoras]
        self.y_train = dataset_train.loc[:,'property_price']
        self.x_test_original = None
        self.x_train_original = None
        self.precio_maximo = max(max(dataset_test['property_price']), max(dataset_train['property_price']))
        self.escalar_datasets()
        self.entrenar_modelo()
    
    def escalar_datasets(self):
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(self.x_train))
        self.x_train_original = self.x_train
        self.x_test_original = self.x_test
        self.x_train=sscaler.transform(pd.DataFrame(self.x_train))
        self.x_test=sscaler.transform(pd.DataFrame(self.x_test))
            
        self.y_train = self.y_train / self.precio_maximo
        

    def entrenar_modelo(self):
    # Creo un modelo Red Neuronal
        d_in= len(self.variables_predictoras)
        d_out=1

        capas = [
            # input layer
            keras.layers.Dense(d_in,input_shape=(d_in,), activation= self.funcion_activacion)
        ]

        #inserta capas que van decreciendo en cantidad de neuronas
        for i in range(self.cantidad_primera_capa,2,-1):
            capas.append(keras.layers.Dense(i, activation= self.funcion_activacion ))

        capas.append(keras.layers.Dense(d_out, ))
        self.modelo = keras.Sequential(capas)

        self.modelo.compile(
        optimizer= self.optimizador(self.learning_rate), 
        loss=self.metrica, 
        metrics=[self.metrica], 
        )

        self.modelo.fit(self.x_train,self.y_train,epochs=self.epoch,batch_size=self.batch_size,verbose=False)
    
    def plotear(self):
        plot_regression(self.modelo,self.x_test,self.y_test,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        
    def testear_modelo(self):
        y_pred=self.modelo.predict(self.x_test)
        for i in y_pred:
            i[0] = i[0] * self.precio_maximo
        mse=mean_squared_error(self.y_test,y_pred)
        self.resultado = mse ** (1/2)
        return self.resultado
    
    def predecir(self, df):

        df_para_predecir =  df.loc[:,self.variables_predictoras]
        df_precios = df.loc[:,'property_price']
        
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(df_para_predecir))
        df_para_predecir=sscaler.transform(pd.DataFrame(df_para_predecir))

        
        y_pred=self.modelo.predict(df_para_predecir)
        for i in y_pred:
            i[0] = i[0] * self.precio_maximo

        mse=mean_squared_error(df_precios,y_pred)
        resultado = mse ** (1/2)
        #plot_regression(self.modelo,df_para_predecir,df_precios,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        return y_pred , resultado
    
    def info(self):
        print('\n','\n')
        print('\n',
        self.variables_predictoras , '\n',
        self.cantidad_primera_capa ,'\n',
        self.funcion_activacion ,'\n',
        self.optimizador ,'\n',
        self.learning_rate ,'\n',
        self.metrica ,'\n',
        self.epoch ,'\n',
        self.batch_size ,'\n')

definimos los conjuntos de hiperparametros que queremos que pruebe

In [ ]:
columnas_importantes =  ['latitud', 'longitud', 'property_surface_total', 'property_surface_covered']
columnas_extras = []
for e in columnas_predictoras:
    if e not in columnas_importantes:
        columnas_extras.append(e)       

def agregar_columnas(posibles_variables_predictoras, columnas, cant_columnas):
            i = 0
            while (i < cant_columnas): 
                pos = random.randrange(i,len(columnas), 1)
                posibles_variables_predictoras.append(columnas[pos])
                columnas[i], columnas[pos] = columnas[pos], columnas[i]
                i += 1

posibles_cantidad_de_primera_capa = []
for i in range(1,20):
    posibles_cantidad_de_primera_capa.append(i)
                                        #vimos que los modelos buenos utilizaban "relu" asique dejamos de utilizar el resto dentro de random search
posibles_funcion_activacion = ["relu","tanh", "softmax", "sigmoid"]
posibles_metricas = ['mse', 'mae']

posibles_learning_rate = [0.1, 0.01]

In [ ]:
def funcion_optimizadoraNADAM(learnigRate):
            return keras.optimizers.Nadam(learning_rate = learnigRate)

def funcion_optimizadoraRMSPROP(learnigRate):
            return keras.optimizers.RMSprop(learning_rate = learnigRate)

def funcion_optimizadoraADAMAX(learnigRate):
            return keras.optimizers.Adamax(learning_rate = learnigRate)

def funcion_optimizadoraADADELTA(learnigRate):
            return keras.optimizers.Adadelta(learning_rate = learnigRate)

posibles_optimizador = [funcion_optimizadoraNADAM, funcion_optimizadoraRMSPROP, funcion_optimizadoraADAMAX, funcion_optimizadoraADADELTA]

In [ ]:
def obtener_posibles_variables_predictoras():
        posibles_variables_predictoras = []     
        cant_columnas_importantes = random.randrange(1,len(columnas_importantes)+1, 1)
        cant_columnas_extras = random.randrange(0,len(columnas_extras)+1, 1)

        agregar_columnas(posibles_variables_predictoras, columnas_importantes, cant_columnas_importantes)
        agregar_columnas(posibles_variables_predictoras, columnas_extras, cant_columnas_extras)
        return posibles_variables_predictoras

crea, entrena y luego predice un modelo con hiperparametros randomizados sobre datasets peque;os para reducir el tiempo de computo y obtener buenos hiperparametros

In [ ]:
def random_validator(cantidad_modelos, posibles_cantidad_de_primera_capa, posibles_funcion_activacion, posibles_optimizador, 
    posibles_learning_rate, posibles_metricas, obtener_posibles_variables_predictoras):
    
    resultados = []
    for i in range(cantidad_modelos):

        variables_predictoras = obtener_posibles_variables_predictoras()
        cantidad_de_primera_capa = random.choice(posibles_cantidad_de_primera_capa)
        funcion_activacion = random.choice(posibles_funcion_activacion)
        optimizador = random.choice(posibles_optimizador)
        learning_rate = random.choice(posibles_learning_rate)
        metrica = random.choice(posibles_metricas)
        epoch = random.randrange(100, 300, 10)
        batch_size = random.randrange(10, 1000, 1)
        df_reducido = viviendas_train[:3000]#random.randrange(3000, 5000, 1)
        df_train, df_test = train_test_split(df_reducido.loc[:],test_size=0.2)

        print(variables_predictoras)

        print('\n','\n')
        print(i , '\n',
        variables_predictoras , '\n',
        cantidad_de_primera_capa ,'\n',
        funcion_activacion ,'\n',
        optimizador ,'\n',
        learning_rate ,'\n',
        metrica ,'\n',
        epoch ,'\n',
        batch_size ,'\n',)

        modelo = ModeloRedes(df_train, df_test, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size)
        resultados.append([modelo, modelo.testear_modelo()])
        print(modelo.testear_modelo())
    return resultados

In [ ]:
#Hace 50 modelos con hiperparametros random random
def func():
    resultados = random_validator(50, posibles_cantidad_de_primera_capa, posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas,obtener_posibles_variables_predictoras)

    return resultados

In [ ]:
#dado un conjunto de modelos se queda con los modelos cuyo rmse sea menor a 130000
def obtener_mejores(resultados):
    mejores = []
    for i in resultados:
        if (i[1] < 130000):
            mejores.append(i)

    return mejores

Entrenamos 150 modelos

In [ ]:
mejores = []

In [ ]:
resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:
resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:
for i in mejores:
    for j in i:
        print(j)
        j[0].info()

Entrenamos modelos con los hiperparametros obtenidos en los datasets mas chicos sobre todo el dataset

In [ ]:
#reentrena un modelo con los mismo hiperparametros a ver si con una semilla mejor mejora la metrica
def reentrenar_tot(modelo):
    #modelo.info()
    mejor_modelo = ModeloRedes(
        viviendas_train, 
        viviendas_test, 
        modelo.variables_predictoras, 
        modelo.cantidad_primera_capa, 
        modelo.funcion_activacion, 
        modelo.optimizador, 
        modelo.learning_rate, 
        modelo.metrica, 
        modelo.epoch, 
        modelo.batch_size)
        
    mejor_modelo.testear_modelo()
    mejor_resultado = mejor_modelo.resultado 
   
    if mejor_resultado > 150000:
        return None

    for i in range(2):
        print('valor a vencer', mejor_resultado)
        modelo_aux = ModeloRedes(
            viviendas_train, 
            viviendas_test, 
            modelo.variables_predictoras, 
            modelo.cantidad_primera_capa, 
            modelo.funcion_activacion, 
            modelo.optimizador, 
            modelo.learning_rate, 
            modelo.metrica, 
            modelo.epoch, 
            modelo.batch_size)

        resultado_aux = modelo_aux.testear_modelo()
        print('reentrenando intento', i+1)
        resultado_aux = modelo_aux.testear_modelo()
        print(resultado_aux)
        mejor_modelo.info()
        
        if resultado_aux < mejor_resultado:
            mejor_resultado = resultado_aux
            mejor_modelo = modelo_aux 
    return [mejor_modelo, mejor_resultado]

In [ ]:
tots = []

In [ ]:
for i in mejores:
    for j in i:
        tots.append(reentrenar_tot(j[0]))

In [ ]:
print(tots)

In [ ]:
mejor_modelo = tots[0]
for i in tots:
    if (i != None) and (i[1] < mejor_modelo[1]):
        mejor_modelo = i



In [ ]:
mejor_modelo[0].testear_modelo()

In [ ]:
joblib.dump(mejor_modelo[0], './Models/mejor_modelo_ultima_tanda') 

In [ ]:
#mejor_modelo = joblib.load('./Models/mejor_modelo_ultima_tanda')

El mejor modelo obtenido hasta ahora con estas tecnicas es:

In [ ]:
mejor_modelo = joblib.load('./Models/modelo_regresion_103k')

In [ ]:
#ModeloRedes(df_train, df_test, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size)

no_te_queremos_perder_hermoso = ModeloRedes(
    viviendas_train, 
    viviendas_test, 
    mejor_modelo.variables_predictoras, 
    mejor_modelo.cantidad_primera_capa, 
    mejor_modelo.funcion_activacion, 
    mejor_modelo.optimizador, 
    mejor_modelo.learning_rate, 
    mejor_modelo.metrica, 
    mejor_modelo.epoch, 
    mejor_modelo.batch_size)

In [ ]:
no_te_queremos_perder_hermoso.testear_modelo()

hasat aca corre

In [ ]:
mejor_modelo.plotear()

In [ ]:
# Performance
performance_test = pd.DataFrame({'Valor Real': y_test_knn,
                            'Prediccion': y_pred_test_knn.round(),
                            'Error': y_test_knn - y_pred_test_knn.round()})
# View

performance_test.head()

In [ ]:
reentrenado = ModeloRedes(viviendas, 
    mejor_modelo.variables_predictoras, 
    mejor_modelo.cantidad_primera_capa, 
    mejor_modelo.funcion_activacion, 
    mejor_modelo.optimizador, 
    mejor_modelo.learning_rate, 
    mejor_modelo.metrica, 
    mejor_modelo.epoch, 
    mejor_modelo.batch_size, 
    viviendas.size)

In [ ]:
mejor_modelo.x_train.shape

In [ ]:
mejor_modelo.y_test

In [ ]:
datos_test = pd.DataFrame(mejor_modelo.x_test)
b = pd.DataFrame(mejor_modelo.y_test)
datos_test.join(b)

In [ ]:
mejor_modelo.predecir(datos_test)

In [ ]:
test = pd.concat(mejor_modelo.x_test, mejor_modelo.x_train)
mejor_modelo.predecir(mejor_modelo.x_test)

In [ ]:
mejor_modelo = joblib.load('./Models/modelo_regresion_103k')
        
y_pred=mejor_modelo.modelo.predict(mejor_modelo.x_train)

aux = []
for i in range(len(y_pred)):
     aux.append(y_pred[i][0])  
"""
y_pred = pd.Series(aux)"""
y_pred = aux

performance_train = pd.DataFrame({'Valor Real': mejor_modelo.y_train,
                            'Prediccion': y_pred,
                            'Error': mejor_modelo.y_train - y_pred
                             })

performance_train.head()
performance_train.plot.scatter(x="Valor Real", y="Prediccion", s=10, c='tab:blue').set(title="Valor Real vs Predicción",xlabel='Valor Real',ylabel='Prediccion')

In [ ]:
performance_train.head()

In [ ]:
print('Metrica rmse', mejor_modelo.testear_modelo())

mejor_modelo.info()

In [ ]:
#joblib.dump(model_tot, './Models/red_regresion_108k_tot') 